In [1]:
import sys, os
sys.path.append(os.path.realpath(".."))
os.environ["TF_XLA_FLAGS"]="--tf_xla_cpu_global_jit"

import util_funcs
from importlib import reload
import data_reader as read
import ensembleReader as er
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import constants
import clinical_text_analysis as cta
import tsfresh
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from os import path
import keras_models.dataGen as dg
import keras_models.cnn_models as cnn_models
reload(dg)
import predictGenderConvExp as pg
from addict import Dict
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import keras
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, Activation, Conv2D, Concatenate, Dropout, MaxPool2D, Conv3D, Flatten, LeakyReLU, BatchNormalization

In [3]:
files, genders = cta.demux_to_tokens(cta.getGenderAndFileNames("combined", "01_tcp_ar", True))

In [4]:
sessions = [read.parse_edf_token_path_structure(file)[2] for file in files]
patients = [read.parse_edf_token_path_structure(file)[1] for file in files]

In [5]:
allPatients = list(set(patients))

In [6]:
allPatients.sort()

In [7]:
len(allPatients)

318

In [8]:
sessionDict = Dict()
for i, file in enumerate(files):
    sessionDict[patients[i]][sessions[i]][i].file = file  
    sessionDict[patients[i]][sessions[i]][i].gender = genders[i]
    sessionDict[patients[i]][sessions[i]][i].patient = allPatients.index(patients[i])


In [9]:
#delete all patients with only one session
for patient in set(patients):
    print(len(sessionDict[patient].keys()))    
    if len(sessionDict[patient].keys()) < 2:
        del sessionDict[patient]


5
1
1
1
3
1
1
1
1
1
1
1
2
6
1
5
3
1
3
2
5
1
1
2
2
2
3
1
1
1
2
1
1
1
1
1
1
1
1
1
2
3
1
1
2
2
4
3
1
1
2
1
2
1
1
1
3
1
1
1
1
1
1
2
4
1
6
1
1
1
1
29
3
1
1
1
2
1
7
1
1
2
1
2
2
4
1
1
4
1
1
1
6
2
2
1
2
1
3
1
1
1
5
1
1
1
1
1
7
3
3
8
1
1
6
2
3
1
2
11
5
2
1
4
1
2
9
1
1
2
9
4
1
2
1
1
6
2
3
1
4
4
5
1
1
2
1
7
1
4
3
2
6
3
1
3
1
2
4
4
4
3
4
1
1
2
1
1
9
2
3
1
1
1
3
1
2
1
6
2
1
1
5
1
10
1
2
1
1
3
4
1
1
9
1
1
1
1
6
1
1
1
1
4
13
1
5
1
1
1
11
1
3
2
8
1
1
1
3
1
1
1
5
1
1
2
1
1
1
2
1
6
1
1
2
6
1
1
2
1
1
1
1
4
1
3
1
1
2
1
1
9
1
3
1
5
2
1
1
1
1
1
1
1
1
8
1
1
1
1
1
1
1
1
1
2
3
4
1
10
2
1
1
1
8
7
1
1
2
4
1
1
4
2
2
2
4
1
1
1
1
1
1
1
1
6
4
3
1
1
2
1
1
1
2
10
4
1


In [10]:
def returnFilesAndLabelsFromSessionDict(d):
    files = []
    genders = []
    for id_num in d.keys():
        files.append(d[id_num].file)
        genders.append((d[id_num].gender, d[id_num].patient))
    return files, genders

In [11]:
testPatientFiles = []
testLabels = []
trainPatientFiles = []
trainLabels = []
for patient in sessionDict.keys():
    testSessionToAdd = np.random.choice(list(sessionDict[patient].keys()))
    for session in sessionDict[patient].keys():
        files, genders = returnFilesAndLabelsFromSessionDict(sessionDict[patient][session])
        if session == testSessionToAdd:
            testPatientFiles += files
            testLabels += genders
        else:
            trainPatientFiles += files
            trainLabels += genders

In [12]:
len(testPatientFiles), len(trainPatientFiles)

(681, 2303)

In [13]:
len(allPatients)

318

In [14]:
len(genders)

10

In [15]:
reload(er)
testEnsembler = er.EdfDatasetEnsembler("combined", "01_tcp_ar", max_num_samples=5, edf_tokens=testPatientFiles, n_process=4, labels=testLabels)[:]
trainEnsembler = er.EdfDatasetEnsembler("combined", "01_tcp_ar", max_num_samples=5, edf_tokens=trainPatientFiles, n_process=4, labels=trainLabels)[:]

Starting 4 processes
retrieving: 0
retrieving: 10
retrieving: 20
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 8, less than 9 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 8, less than 9 requested!!!
retrieving: 30
read 1020, less than 1030 requested!!!
read 8, less than 9 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
re

read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030

read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
retrieving: 1620
read 8, less than 9 requested!!!
read 8, less than 9 requested!!!
read 8, less than 9 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 

read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
retrieving: 2270
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 2039, less than 2054 requested!!!
read 203

read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
retrieving: 2880
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 1023, less than 1030 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
retrieving: 2890
retrieving: 2900
retrieving: 2910
retrieving: 2920
read 8, less than 9 requested!!!
rea

...

read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
retrieving: 7870
retrieving: 7880
retrieving: 7890
retrieving: 7900
retrieving: 7910
retrieving: 7920
retrieving: 7930
retrieving: 7940
retrieving: 7950
read 8, less than 9 requested!!!
read 8, less than 9 requested!!!
read 8, less than 9 requested!!!
retrieving: 7960
retrieving: 7970
retrieving: 7980
retrieving: 7990
retrieving: 8000
retrieving: 8010
retrieving: 8020
retrieving: 8030
retrieving: 8040
retrieving: 8050
retrieving: 8060
retrieving: 8070
read 6, less than 9 requested!!!
read 6, less than 9 requested!!!
read 6, less than 9 requested!!!
retrieving: 8080
retrieving: 8090


read 1007, less than 1030 requested!!!
read 1007, less than 1030 requested!!!
read 1007, less than 1030 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
retrieving: 8660
retrieving: 8670
retrieving: 8680
retrieving: 8690
retrieving: 8700
retrieving: 8710
retrieving: 8720
retrieving: 8730
read 8, less than 9 requested!!!
read 8, less than 9 requested!!!
read 8, less than 9 requested!!!
retrieving: 8740
retrieving: 8750
retrieving: 8760
retrieving: 8770
retrieving: 8780
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested!!!
read 1020, less than 1030 requested

read 8, less than 9 requested!!!
read 8, less than 9 requested!!!
read 8, less than 9 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
retrieving: 9400
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 

retrieving: 10240
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 1021, less than 1030 requested!!!
read 10

read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 1024, less than 1030 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
read 4, less than 9 requested!!!
retrieving: 11050
retrieving: 11060
retrieving: 11070
retrieving: 11080
retrieving: 11090
retrieving: 11100
retrieving: 11110
retrieving: 11120
retrieving: 11130
retrieving: 11140
retrieving: 11150
retrieving: 11160
retrieving: 11170
retrieving: 11180
retrieving: 11190
retrieving: 11200
retrieving: 11210
retrieving: 11220
retrieving: 11230
retrieving: 11240
retrieving: 11250
retrieving: 11260
retrieving: 11270
retrieving: 11280
retrieving: 11290
retrieving: 11300
retr

In [16]:
len(trainEnsembler)

11496

In [17]:
trainEnsembler, validEnsembler = train_test_split(trainEnsembler, test_size=0.1)

In [18]:
reload(dg)
testDataGen = dg.DataGenMultipleLabels(testEnsembler, batch_size=256, num_labels=2, n_classes=(2, len(allPatients)), precache=True, time_first=False)
validDataGen = dg.DataGenMultipleLabels(validEnsembler, batch_size=256, num_labels=2, n_classes=(2, len(allPatients)), precache=True, time_first=False)
trainDataGen = dg.DataGenMultipleLabels(trainEnsembler, batch_size=64, num_labels=2, n_classes=(2, len(allPatients)), precache=True, time_first=False, shuffle=False)

In [19]:
len(testDataGen), len(trainDataGen)

(14, 162)

In [20]:
x, y = testDataGen[0]

In [21]:
x.shape

(256, 21, 500, 1)

In [22]:
y

[array([[0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0

In [108]:
x, y = cnn_models.inception_like_pre_layers(input_shape=(21, 500, 1), num_filters=20, dropout=0.5, num_layers=4)

In [109]:
y_gender = Dense(2, activation="softmax", name="gender")(y)
y_patient = Dense(len(allPatients), activation="softmax", name="patient")(y)

In [110]:
model = Model(inputs=x, outputs=[y_gender, y_patient])

In [111]:
model.summary()
model.save("baseModel.h5")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 21, 500, 1)   0                                            
__________________________________________________________________________________________________
conv2d_125 (Conv2D)             (None, 20, 499, 10)  50          input_13[0][0]                   
__________________________________________________________________________________________________
conv2d_129 (Conv2D)             (None, 19, 498, 10)  100         input_13[0][0]                   
__________________________________________________________________________________________________
conv2d_133 (Conv2D)             (None, 18, 497, 10)  170         input_13[0][0]                   
__________________________________________________________________________________________________
conv2d_137

In [112]:
from keras.utils import multi_gpu_model
model = multi_gpu_model(model, 4)

In [113]:
adam = keras.optimizers.Adam(lr=0.0005)
model_gender_save = keras.callbacks.ModelCheckpoint("genderPredictor.h5", monitor="val_gender_loss", save_best_only=True, mode="min", verbose=True)
model_patient_save = keras.callbacks.ModelCheckpoint("patientPredictor.h5", monitor="val_patient_loss", save_best_only=True, mode="min", verbose=True)


earlyStopsOnGender = keras.callbacks.EarlyStopping(monitor="val_gender_loss", mode="min", patience=5, verbose=True)
earlyStopsOnPatient = keras.callbacks.EarlyStopping(monitor="val_patient_loss", mode="min", patience=5, verbose=True)
cb = [model_gender_save, model_patient_save, earlyStopsOnPatient]

model.compile(adam, loss=["categorical_crossentropy", "categorical_crossentropy"], metrics=["categorical_accuracy"], loss_weights=[0.5, 1.5])

In [114]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 21, 500, 1)   0                                            
__________________________________________________________________________________________________
lambda_9 (Lambda)               (None, 21, 500, 1)   0           input_13[0][0]                   
__________________________________________________________________________________________________
lambda_10 (Lambda)              (None, 21, 500, 1)   0           input_13[0][0]                   
__________________________________________________________________________________________________
model_15 (Model)                [(None, 2), (None, 3 15785770    lambda_9[0][0]                   
                                                                 lambda_10[0][0]                  
__________

In [115]:
history = model.fit_generator(trainDataGen, epochs=100, validation_data=validDataGen, callbacks=cb)

Epoch 1/100
162/162 [==============================] - 30s 185ms/step - loss: 8.4132 - gender_loss: 1.0689 - patient_loss: 5.2525 - gender_categorical_accuracy: 0.5682 - patient_categorical_accuracy: 0.0966 - val_loss: 10.3102 - val_gender_loss: 0.6900 - val_patient_loss: 6.6435 - val_gender_categorical_accuracy: 0.6330 - val_patient_categorical_accuracy: 0.0148

Epoch 00001: val_gender_loss improved from inf to 0.68998, saving model to genderPredictor.h5

Epoch 00001: val_patient_loss improved from inf to 6.64346, saving model to patientPredictor.h5
Epoch 2/100
162/162 [==============================] - 9s 56ms/step - loss: 6.6470 - gender_loss: 1.0715 - patient_loss: 4.0742 - gender_categorical_accuracy: 0.6054 - patient_categorical_accuracy: 0.2185 - val_loss: 10.0612 - val_gender_loss: 0.6953 - val_patient_loss: 6.4757 - val_gender_categorical_accuracy: 0.6435 - val_patient_categorical_accuracy: 0.1009

Epoch 00002: val_gender_loss did not improve from 0.68998

Epoch 00002: val_pat

In [116]:
genderPredictor = keras.models.load_model("genderPredictor.h5")
genderPredictor = multi_gpu_model(genderPredictor, 2)

In [117]:
len(testEnsembler)

3396

In [118]:
ypred = genderPredictor.predict(np.stack([data[0].reshape(500,21,1).transpose(1,0,2) for data in testEnsembler]))

In [119]:
genderPred = ypred[0]

In [120]:
accuracy_score(genderPred.argmax(1), [data[1][0] for data in testEnsembler])

0.5371024734982333

In [121]:
accuracy_score(ypred[1].argmax(1), [data[1][1] for data in testEnsembler])

0.034746760895170786

In [122]:
roc_auc_score(genderPred.argmax(1), [data[1][0] for data in testEnsembler])

0.5767726439769285

In [123]:
class_weights = [(0,1),(1,100), (1,75), (1,50), (1,25), (1, 10), (1,5), (1,1)]
class_weights += [(weight[1], weight[0]) for weight in class_weights]
class_weights = class_weights[:-1]
class_weights
# class_weights = list(set(class_weights))


[(0, 1),
 (1, 100),
 (1, 75),
 (1, 50),
 (1, 25),
 (1, 10),
 (1, 5),
 (1, 1),
 (1, 0),
 (100, 1),
 (75, 1),
 (50, 1),
 (25, 1),
 (10, 1),
 (5, 1)]

In [127]:
trainDataGen.batch_size = 128

In [ ]:
results = Dict()
for class_weight in class_weights:
    print("starting {}".format(class_weight))
    model = keras.models.load_model("baseModel.h5")
    model = multi_gpu_model(model, 4)
    adam = keras.optimizers.Adam(lr=0.001)
    model_save = keras.callbacks.ModelCheckpoint("lossPredictor{}.h5".format(class_weight), monitor="val_loss", save_best_only=True, mode="min", verbose=True)
    early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5, verbose=True)
    cb = [model_save, early_stopping]
    model.compile(adam, loss=["categorical_crossentropy", "categorical_crossentropy"], metrics=["categorical_accuracy"], loss_weights=list(class_weight))
    history = model.fit_generator(trainDataGen, epochs=100, validation_data=validDataGen, callbacks=cb)
    results[class_weight].history = history.history
    model = keras.models.load_model("lossPredictor{}.h5".format(class_weight))
    model = multi_gpu_model(model, 2)
    ypred = model.predict(np.stack([data[0].reshape(500,21,1).transpose(1,0,2) for data in testEnsembler]))
    results[class_weight].gender.acc = accuracy_score(ypred[0].argmax(1), [data[1][0] for data in testEnsembler])
    results[class_weight].gender.AUC = roc_auc_score(ypred[0].argmax(1), [data[1][0] for data in testEnsembler])
    results[class_weight].patient.acc = accuracy_score(ypred[1].argmax(1), [data[1][1] for data in testEnsembler])
#     results[class_weight].patient.AUC = roc_auc_score(ypred[1].argmax(1), [data[1][1] for data in testEnsembler])
import pickle as pkl
pkl.dump(results, open("results.pkl", 'wb'))

starting (0, 1)


/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Epoch 1/100
81/81 [==============================] - 117s 1s/step - loss: 5.4210 - gender_loss: 0.9878 - patient_loss: 5.4210 - gender_categorical_accuracy: 0.4978 - patient_categorical_accuracy: 0.0969 - val_loss: 8.2152 - val_gender_loss: 0.9557 - val_patient_loss: 8.2152 - val_gender_categorical_accuracy: 0.3930 - val_patient_categorical_accuracy: 0.0096

Epoch 00001: val_loss improved from inf to 8.21519, saving model to lossPredictor(0, 1).h5
Epoch 2/100
81/81 [==============================] - 8s 95ms/step - loss: 4.3710 - gender_loss: 0.9172 - patient_loss: 4.3710 - gender_categorical_accuracy: 0.5116 - patient_categorical_accuracy: 0.2126 - val_loss: 6.1779 - val_gender_loss: 0.8109 - val_patient_loss: 6.1779 - val_gender_categorical_accuracy: 0.3887 - val_patient_categorical_accuracy: 0.1165

Epoch 00002: val_loss improved from 8.21519 to 6.17786, saving model to lossPredictor(0, 1).h5
Epoch 3/100
81/81 [==============================] - 8s 95ms/step - loss: 3.5672 - gender_lo

/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Epoch 1/100
81/81 [==============================] - 138s 2s/step - loss: 539.5699 - gender_loss: 1.2305 - patient_loss: 5.3834 - gender_categorical_accuracy: 0.5744 - patient_categorical_accuracy: 0.0998 - val_loss: 790.2141 - val_gender_loss: 0.7896 - val_patient_loss: 7.8942 - val_gender_categorical_accuracy: 0.6174 - val_patient_categorical_accuracy: 0.0122

Epoch 00001: val_loss improved from inf to 790.21407, saving model to lossPredictor(1, 100).h5
Epoch 2/100
81/81 [==============================] - 8s 97ms/step - loss: 427.0662 - gender_loss: 1.1741 - patient_loss: 4.2589 - gender_categorical_accuracy: 0.6090 - patient_categorical_accuracy: 0.2192 - val_loss: 687.0864 - val_gender_loss: 0.7787 - val_patient_loss: 6.8631 - val_gender_categorical_accuracy: 0.5296 - val_patient_categorical_accuracy: 0.0252

Epoch 00002: val_loss improved from 790.21407 to 687.08643, saving model to lossPredictor(1, 100).h5
Epoch 3/100
81/81 [==============================] - 8s 96ms/step - loss: 

/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Epoch 1/100
81/81 [==============================] - 159s 2s/step - loss: 412.9058 - gender_loss: 1.2715 - patient_loss: 5.4885 - gender_categorical_accuracy: 0.5655 - patient_categorical_accuracy: 0.0988 - val_loss: 771.8737 - val_gender_loss: 0.9885 - val_patient_loss: 10.2785 - val_gender_categorical_accuracy: 0.4443 - val_patient_categorical_accuracy: 0.0148

Epoch 00001: val_loss improved from inf to 771.87371, saving model to lossPredictor(1, 75).h5
Epoch 2/100
81/81 [==============================] - 8s 100ms/step - loss: 333.2595 - gender_loss: 1.1830 - patient_loss: 4.4277 - gender_categorical_accuracy: 0.6045 - patient_categorical_accuracy: 0.2067 - val_loss: 614.4362 - val_gender_loss: 0.9201 - val_patient_loss: 8.1802 - val_gender_categorical_accuracy: 0.6217 - val_patient_categorical_accuracy: 0.0487

Epoch 00002: val_loss improved from 771.87371 to 614.43619, saving model to lossPredictor(1, 75).h5
Epoch 3/100
81/81 [==============================] - 8s 100ms/step - loss:

/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Epoch 1/100
81/81 [==============================] - 187s 2s/step - loss: 268.5224 - gender_loss: 1.1801 - patient_loss: 5.3468 - gender_categorical_accuracy: 0.5783 - patient_categorical_accuracy: 0.1072 - val_loss: 375.2940 - val_gender_loss: 1.4336 - val_patient_loss: 7.4772 - val_gender_categorical_accuracy: 0.4104 - val_patient_categorical_accuracy: 0.0235

Epoch 00001: val_loss improved from inf to 375.29400, saving model to lossPredictor(1, 50).h5
Epoch 2/100
81/81 [==============================] - 8s 103ms/step - loss: 209.4822 - gender_loss: 1.1442 - patient_loss: 4.1668 - gender_categorical_accuracy: 0.6041 - patient_categorical_accuracy: 0.2287 - val_loss: 289.3949 - val_gender_loss: 1.4449 - val_patient_loss: 5.7590 - val_gender_categorical_accuracy: 0.6226 - val_patient_categorical_accuracy: 0.1104

Epoch 00002: val_loss improved from 375.29400 to 289.39491, saving model to lossPredictor(1, 50).h5
Epoch 3/100
81/81 [==============================] - 8s 102ms/step - loss: 

In [133]:
results[(0,1)].history.history

{'val_loss': [8.215194356337838,
  6.177855849058732,
  5.655041023751964,
  6.375176536725915,
  6.021530593374501,
  5.7514516755808955,
  4.19344938609911,
  5.564590275805929,
  4.7920146021635635,
  3.781208082696666,
  4.582918354531993,
  4.381876838518226,
  4.06437245990919,
  4.376558230026909,
  3.8648587707851245],
 'val_gender_loss': [0.9556532542601875,
  0.810909714491471,
  0.7457960929041323,
  0.7778411540777787,
  0.7408455403991367,
  0.7510165415639463,
  0.727168431074723,
  0.6696126983476721,
  0.6908006458697111,
  0.7111556276031162,
  0.6936395059461179,
  0.7119261082358982,
  0.7224195516627768,
  0.7146161087699558,
  0.7168778117843296],
 'val_patient_loss': [8.215194356337838,
  6.177855849058732,
  5.655041023751964,
  6.375176536725915,
  6.021530593374501,
  5.7514516755808955,
  4.19344938609911,
  5.564590275805929,
  4.7920146021635635,
  3.781208082696666,
  4.582918354531993,
  4.381876838518226,
  4.06437245990919,
  4.376558230026909,
  3.86485